# Use selenium to scrape carmax.com

In [391]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup

import time
import os
import numpy as np

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)

### Scape the list of URLs

In [16]:
URL = "https://www.carmax.com/search"
driver.get(URL)
time.sleep(1);
link_lists = []

max_value = driver.find_element_by_xpath('//div[@data-max-price="100000"]')
time.sleep(1)
max_value.send_keys(Keys.DOWN)
time.sleep(0.5)
max_value.send_keys(Keys.DOWN)
time.sleep(0.5)
max_value.send_keys(Keys.DOWN)
max_value.send_keys(Keys.RETURN)
#max_value.click()

date_element = driver.find_element_by_xpath('//a[starts-with(@href, "/car/")]')
date_element.get_attribute('href')

a href="/car/16146206"

In [17]:
link_elements = driver.find_elements_by_xpath('//h3[@class="vehicle-browse--result-title"]/a')
link_lists = [element.get_attribute('href') for element in link_elements]

In [18]:
link_lists

['https://www.carmax.com/car/15879910',
 'https://www.carmax.com/car/16061803',
 'https://www.carmax.com/car/15824438',
 'https://www.carmax.com/car/15165784',
 'https://www.carmax.com/car/15210066',
 'https://www.carmax.com/car/15824549',
 'https://www.carmax.com/car/15867329',
 'https://www.carmax.com/car/15380380',
 'https://www.carmax.com/car/15867035',
 'https://www.carmax.com/car/15944508',
 'https://www.carmax.com/car/16169822',
 'https://www.carmax.com/car/15086949',
 'https://www.carmax.com/car/15585428',
 'https://www.carmax.com/car/15895613',
 'https://www.carmax.com/car/15777631',
 'https://www.carmax.com/car/16057238',
 'https://www.carmax.com/car/15867327',
 'https://www.carmax.com/car/16056848',
 'https://www.carmax.com/car/15867099',
 'https://www.carmax.com/car/15867057']

In [20]:
endofresult = False

In [21]:
while(endofresult == False):
    try:
        driver.find_element_by_css_selector(".pagination--next-disable")
        endofresult = True
        print ("This is the end of search result!")
    except Exception as e:
        next_element = driver.find_element_by_class_name("pagination--next")
        next_element.click()
        time.sleep(2)
        link_elements = driver.find_elements_by_xpath('//h3[@class="vehicle-browse--result-title"]/a')
        link_lists = link_lists + [element.get_attribute('href') for element in link_elements]

This is the end of search result!


In [23]:
len(link_lists)

156

### Scape the individual Car main pages

In [ ]:
driver = webdriver.Chrome(chromedriver)
driver.get("https://www.carmax.com/car/16061803")

In [300]:
def find_classes(URL, classname_function_dict):
    driver.get(URL)
    review_button = driver.find_element_by_xpath('//a[@class="kmx-button kmx-button--secondary kmx-button--large kmx-button--raised"]')
    review_button.click()
    result = {}
    for classname in sorted(classname_function_dict.keys()):
        result[classname] = driver.find_elements_by_class_name(classname)
    return result

In [313]:
testurl = 'https://www.carmax.com/car/16061803'
testurl = 'https://www.carmax.com/car/16104605'

In [314]:
classname_function_dict = {
    "car-page-header__car-title__year-make":process_year_make,
    "car-page-header__car-title__model-trim":process_min,
    "price-mileage--value":process_mileage_value,
    "action-bar--image-counter":process_image_count,
    "store-information--location":process_min,
    "about-this-car--header__subtitle":process_stock_vin,
    "gas-mileage-container":process_city_highway,
    "kmx-table":process_base_spec,
    "linear-rating":process_user_rating
}

In [315]:
result = find_classes(testurl, classname_function_dict)

In [230]:
def process_mileage_value(input):
    price = int(input[0]
                 .text
                 .replace('$','')
                 .replace('*','')
                 .replace(',',''))
    mileage = int(input[1]
                 .text
                 .replace('K',''))
    return [price, mileage]

In [243]:
def process_year_make(input):
    splited = input[0].text.split(' ')
    year = splited[0]
    brand = splited[1]
    return [year, brand]

In [232]:
def process_min(input):
    return [input[0].text]

In [248]:
def process_image_count(input):
    return [int(input[0].text.split(" ")[2])]

In [234]:
def process_stock_vin(input):
    splited = input[0].text.split("|")
    stock = splited[0].split(" ")[2]
    vin = splited[1].split(" ")[2]
    return [stock, vin]

In [249]:
def process_city_highway(input):
    splited = input[0].text.split("\n")
    city = int(splited[0])
    highway = int(splited[2])
    return [city, highway]

In [254]:
def process_base_spec(input):
    color = input[0].text.split("\n")
    exterior = color[1].split(" ")[1]
    interior = color[2].split(" ")[1]
    basespec = input[1].text.split("\n")
    transmission = basespec[1].split(" ")[1]
    drive = basespec[2].split(" ")[1]
    engine = float(basespec[3].split(" ")[1].replace("L",""))
    cylinder = int(basespec[4].split(" ")[1])
    horsepower = int(basespec[5].split(" ")[1])
    torque = int(basespec[6].split(" ")[1])
    return [exterior, interior, transmission, drive, engine, cylinder, horsepower, torque]

In [302]:
def process_user_rating(input):
    user_star_5 = int(input[0].text.split("\n")[1])
    user_star_4 = int(input[1].text.split("\n")[1])
    user_star_3 = int(input[2].text.split("\n")[1])
    user_star_2 = int(input[3].text.split("\n")[1])
    user_star_1 = int(input[4].text.split("\n")[1])
    return [user_star_5, user_star_4, user_star_3, user_star_2, user_star_1]

In [303]:
test = []
for value in sorted(result.keys()):
    test = test + (classname_function_dict[value](result[value]))
    print (classname_function_dict[value](result[value]))

['16061803', '19UUA8F5XDA013730']
[23]
['TL']
['2013', 'Acura']
[20, 29]
['White', 'Black', 'Automatic', '2WD', 3.5, 6, 280, 254]
[6, 3, 1, 0, 0]
[18998, 42]
['San Francisco, CA']


In [304]:
test

['16061803',
 '19UUA8F5XDA013730',
 23,
 'TL',
 '2013',
 'Acura',
 20,
 29,
 'White',
 'Black',
 'Automatic',
 '2WD',
 3.5,
 6,
 280,
 254,
 6,
 3,
 1,
 0,
 0,
 18998,
 42,
 'San Francisco, CA']

In [291]:
class_name = "linear-rating"
check = result[class_name]
print (type(check))
print (len(check))
check[0].text

<class 'list'>
0


IndexError: list index out of range

In [362]:
check = driver.find_elements_by_class_name("star-rating__stars")

In [363]:
len(check)

6

In [364]:
check[0].text

'(26)'

In [370]:
date_element = check[0].find_element_by_xpath('//i[@class="m-r-xs"]/i')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//i[@class="m-r-xs"]/i"}
  (Session info: chrome=67.0.3396.99)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.13.5 x86_64)


In [366]:
date_element.text

''

In [324]:
check[1].text

''

In [220]:
check[1].find_elements_by_tag_name("td")

[<selenium.webdriver.remote.webelement.WebElement (session="1a7d3754c297d45a16e992442f4e429e", element="0.7062259897639325-18")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1a7d3754c297d45a16e992442f4e429e", element="0.7062259897639325-19")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1a7d3754c297d45a16e992442f4e429e", element="0.7062259897639325-20")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1a7d3754c297d45a16e992442f4e429e", element="0.7062259897639325-21")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1a7d3754c297d45a16e992442f4e429e", element="0.7062259897639325-22")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1a7d3754c297d45a16e992442f4e429e", element="0.7062259897639325-23")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1a7d3754c297d45a16e992442f4e429e", element="0.7062259897639325-24")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1a7d3754c297d45a16e992442f4e42

In [205]:
(check[0].text).split("\n")

['20', 'City', '29', 'Highway']

In [174]:
splited.split(" ")

['', 'VIN:', '19UUA8F5XDA013730', '']

In [129]:
result["price-mileage--value"][2].text

''

### Scape the recall page

In [371]:
URL = "https://www.nhtsa.gov/recalls?vin=4T1BK36B38U322693"

In [372]:
driver.get(URL)

In [ ]:
By.cssSelector(".alert.alert-success")

In [382]:
time.sleep(4)
check = driver.find_element_by_css_selector(".acsInviteButton.acsDeclineButton")
check.click()

In [385]:
check = driver.find_elements_by_css_selector(".col-sm-8.active-recalls--list")

In [386]:
len(check)

0

### Scape the autocheck page

In [387]:
URL = "https://www.carmax.com/car/16104605/vehicle-history"
driver.get(URL)

In [402]:
check = driver.find_element_by_css_selector("table")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"table"}
  (Session info: chrome=67.0.3396.99)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.13.5 x86_64)


In [392]:

soup = BeautifulSoup(URL, 'html5lib')

/Users/songlin/anaconda2/envs/Python35/lib/python3.5/site-packages/bs4/__init__.py:282: UserWarning: "https://www.carmax.com/car/16104605/vehicle-history" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
